In [2]:
import requests
from bs4 import BeautifulSoup
import urllib.parse
import os
import pandas as pd
import math
import sqlite3

## Scrapping de los datos del dataset TLC Trip Record

In [ ]:
# url donde se encuentran links
url = "https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page"
response = requests.get(url)

# se verifica si la solicitud fue exitosa (codigo de estado 200)
if response.status_code==200:
    #parsear el condido html con beautiful soup
    soup = BeautifulSoup(response.text,"html.parser")

    #Buscar todos los enlaces para el año 2021
    year_section = soup.find('div',{'id':'faq2021'})

    if year_section:
        links = year_section.find_all('a')
        for link in links:
            # obtener url completa del enlace
            link_url = urllib.parse.urljoin(url,link.get('href'))
            
            #se extrae el nombre del archivo del enlace
            file_name = os.path.basename(link_url)

            #se construye la ruta completa del archivo a ruta local
            file_path = os.path.join(output_directory,file_name)
            #se descarga el archivo si no existe aun (el script esta pensado para ser re-ejecutado)
            if not os.path.exists(file_path):
                response_file = requests.get(link_url)

                if response_file.status_code == 200:
                    with open(file_path,'wb') as file:
                        file.write(response_file.content)
                    print(f'archivo descargado: {file_path}')
                else:
                    print(f'error al descargar el archivo desde {link_url}. codigo de estado: {response_file.status_code}')


## Limpieza de los datos del dataset TLC Trip Record Data

### Funciones usadas para persistir los datos en la Base de datos SQLite

In [ ]:
def sql_esquema_tabla(nombre_tabla,tabla_existente=False):
    if not tabla_existente:
        if 'green' in nombre_tabla:
            sql_command = """
            CREATE TABLE IF NOT EXISTS green (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                VendorID INTEGER NOT NULL,
                lpep_pickup_datetime DATETIME,
                lpep_dropoff_datetime DATETIME,
                store_and_fwd_flag VARCHAR(4),
                RatecodeID INTEGER,
                PULocationID INTEGER,
                DOLocationID INTEGER,
                passenger_count INTEGER,
                trip_distance FLOAT,
                fare_amount FLOAT,
                extra FLOAT,
                mta_tax FLOAT,
                tip_amount FLOAT,
                tolls_amount FLOAT,
                improvement_surcharge FLOAT,
                total_amount FLOAT,
                payment_type INTEGER,
                trip_type INTEGER,
                congestion_surcharge FLOAT
            )
            """
        elif 'yellow' in nombre_tabla:
            sql_command = """
            CREATE TABLE IF NOT EXISTS yellow (
                id INTEGER PRIMARY KEY  AUTOINCREMENT,
                VendorID INTEGER NOT NULL,
                tpep_pickup_datetime DATETIME,
                tpep_dropoff_datetime DATETIME,
                passenger_count INTEGER,
                trip_distance FLOAT,
                RatecodeID INTEGER,
                store_and_fwd_flag VARCHAR(4),
                PULocationID INTEGER,
                DOLocationID INTEGER,
                payment_type INTEGER,
                fare_amount FLOAT,
                extra FLOAT,
                mta_tax FLOAT,
                tip_amount FLOAT,
                tolls_amount FLOAT,
                improvement_surcharge FLOAT,
                total_amount FLOAT,
                congestion_surcharge FLOAT,
                Airport_fee INTEGER
            )
            """
        elif 'fhvhv' in nombre_tabla:
            sql_command = """
            CREATE TABLE IF NOT EXISTS fhvhv(
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                hvfhs_license_num VARCHAR(12) NOT NULL,
                dispatching_base_num VARCHAR(12) NOT NULL,
                originating_base_num VARCHAR(12) NOT NULL,
                request_datetime DATETIME,
                on_scene_datetime DATETIME,
                pickup_datetime DATETIME,
                dropoff_datetime DATETIME,
                PULocationID INTEGER,
                DOLocationID INTEGER,
                trip_miles FLOAT,
                trip_time INTEGER,
                base_passenger_fare FLOAT,
                tolls FLOAT,
                bcf FLOAT,
                sales_tax FLOAT,
                congestion_surcharge FLOAT,
                airport_fee FLOAT,
                tips FLOAT,
                driver_pay FLOAT,
                shared_request_flag VARCHAR(1),
                shared_match_flag VARCHAR(1),
                access_a_ride_flag VARCHAR(1),
                wav_request_flag VARCHAR(1),
                wav_match_flag VARCHAR(1))"""
            
        elif 'fhv' in nombre_tabla:
            sql_command= """
            CREATE TABLE IF NOT EXISTS fhv (
                id INTEGER PRIMARY KEY  AUTOINCREMENT,
                dispatching_base_num VARCHAR(12) NOT NULL,
                pickup_datetime DATETIME,
                dropOff_datetime DATETIME,
                PUlocationID INTEGER,
                DOlocationID INTEGER,
                Affiliated_base_number VARCHAR(12) NOT NULL
                )
                """
    return sql_command

# Funcion para cargar archivos
def carga_sqlite(nombre_tabla,comando_sql,ruta_limpia,archivo):
    # comando SQL para la creación de la tabla

    # se conecta a la base de datos SQLlite
    ruta_bd = 'C:\\Users\\Usuario\\Documents\\RENE\\Personal\\Conocimiento\\Capacitacion\\DataScience\\Henry\\ProyecctoFinal\\PT04\\Sprint2\\Data\\BD\\bd.db'
    with sqlite3.connect(ruta_bd) as conn:
        cursor = conn.cursor()

        #si la tabla no existe, crearla con el esquema definido
        cursor.execute(comando_sql)

        df = pd.read_parquet(os.path.join(ruta_limpia,archivo))

        #convertir el dataframe a una tabla SQLite
        df.to_sql(nombre_tabla,conn,if_exists='append',index=False)

### Limpieza

In [30]:
# url donde se encuentran links
url = "https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page"
output_directory = 'C:\\Users\\Usuario\\Documents\\RENE\\Personal\\Conocimiento\\Capacitacion\\DataScience\\Henry\\ProyecctoFinal\\PT04\\Sprint2\\Data\\raw\\TripData'
response = requests.get(url)
ruta_limpia = 'C:\\Users\\Usuario\\Documents\\RENE\\Personal\\Conocimiento\\Capacitacion\\DataScience\\Henry\\ProyecctoFinal\\PT04\\Sprint2\\Data\\clean\\TripData'


descargados = os.listdir(output_directory)

# Iterar sobre los archivos descargados para limpiar y guardar en otra carpeta
for archivo in descargados:
    archivo_ruta = os.path.join(output_directory, archivo)
    archivo_ruta_limpia = os.path.join(ruta_limpia, archivo)
    
    # Se verifica si el archivo existe para no volver a limpiar
    if not os.path.exists(archivo_ruta_limpia):
        #obtener tipo de arcvhio
        tipo_archivo = archivo.split('_')[0].lower()
            
        # Se realiza la limieza según el tipo de archivo (green,yellow,fhv,fhvhv)
        if 'green' == tipo_archivo:
            df = pd.read_parquet(archivo_ruta) 
            # Realizar las operaciones de limpieza para Green trip data
            df.drop('ehail_fee', axis=1, inplace=True)
            df['store_and_fwd_flag'].fillna('N', inplace=True)
            df['RatecodeID'].fillna(1, inplace=True)
            df['RatecodeID'] = df['RatecodeID'].replace(99.0, 1.0)
            redondeo = math.floor(df['passenger_count'].mean())
            df['passenger_count'].fillna(redondeo, inplace=True)
            df = df[df['trip_distance'] <= 250]
            df['payment_type'].fillna(5, inplace=True)
            df['trip_type'].fillna(1, inplace=True)
            df['congestion_surcharge'].fillna(0, inplace=True)
            df['RatecodeID'] = df['RatecodeID'].astype(int)
            df['passenger_count'] = df['passenger_count'].astype(int)
            df['payment_type'] = df['payment_type'].astype(int)
            df['trip_type'] = df['trip_type'].astype(int)


        elif 'yellow' == tipo_archivo:
            df = pd.read_parquet(archivo_ruta)  

            # Realizar las operaciones de limpieza para Yellow trip data
            redondeo = math.floor(df['passenger_count'].mean())
            df['passenger_count'].fillna(redondeo, inplace=True)
            df['RatecodeID'].fillna(1, inplace=True)
            df['RatecodeID'] = df['RatecodeID'].replace(99.0, 1.0)
            df['store_and_fwd_flag'] = df['store_and_fwd_flag'].fillna('N')
            df['payment_type'].replace(0, 5, inplace=True)
            df['congestion_surcharge'].fillna(0, inplace=True)
            if 'airpor_fee' in df.columns:
                df['airport_fee'].fillna(0,inplace=True)
            df['RatecodeID'] = df['RatecodeID'].astype(int)
            df['passenger_count'] = df['passenger_count'].astype(int)
            df['PULocationID'] = df['PULocationID'].astype(int)
            df['DOLocationID'] = df['DOLocationID'].astype(int)
        elif "fhvhv" == tipo_archivo:
            df = pd.read_parquet(archivo_ruta)  

            # Realizar las operaciones de limpieza para FHVHV TRIPDATA
            df['originating_base_num'] = df['originating_base_num'].str.strip()
            comun = dict(df['originating_base_num'].value_counts())
            valor = list(comun.keys())
            df['originating_base_num'].fillna(valor[0], inplace=True)
            # Rellenar valores nulos en access_a_ride_flag
            df['access_a_ride_flag'].replace(" ",'N',inplace=True)
        elif 'fhv' == tipo_archivo:
            df = pd.read_parquet(archivo_ruta)

            # Realizar las operaciones de limpieza para FHV TRIPDATA
            df.drop(df[(df['PUlocationID'].isnull()) & (df['DOlocationID'].isnull())].index, inplace=True)
            df.drop(df[df['PUlocationID'].isnull()].index, inplace=True)
            df.drop(df[df['DOlocationID'].isnull()].index, inplace=True)
            df.drop('SR_Flag', axis=1, inplace=True)
            df['Affiliated_base_number'] = df['Affiliated_base_number'].str.strip()
            comun = dict(df['Affiliated_base_number'].value_counts())
            valor = list(comun.keys())
            df['Affiliated_base_number'] = df['Affiliated_base_number'].replace("", valor[0])
            df['PUlocationID'] = df['PUlocationID'].astype(int)
            df['DOlocationID'] = df['DOlocationID'].astype(int)

        # Guardar en la carpeta "clean"
        df.to_parquet(os.path.join(ruta_limpia, archivo), index=False)
        #se carga el esquema dependiendo del nombre
        sql_esquema = sql_esquema_tabla(tipo_archivo)
        #se carga a la base de datos
        carga_sqlite(tipo_archivo,sql_esquema,ruta_limpia,archivo)


                

## Scrapping del dataset NYC Air Quality

In [ ]:

import requests
import json
import os

url_calidad_aire = "https://data.cityofnewyork.us/resource/c3uy-2p5r.json"
destino = "..\\Data\\aire"
archivo = 'calidad_aire.json'
# Realizar una solicitud GET a la API
response = requests.get(url_calidad_aire)

# Verificar si la solicitud fue exitosa (código de estado 200)
if response.status_code == 200:
    # Los datos están en formato JSON, puedes acceder a ellos utilizando el método json()
    data = response.json()
    
    #si la carpeta no existe, se crea
    if not os.path.exists(destino):
        os.makedirs(destino)

    #construir la ruta completa del archivo dentro de la carpeta destino
    ruta_aire = os.path.join(destino,archivo)    

    # Escribir los datos en un archivo local (por ejemplo, en formato JSON)
    with open(ruta_aire, "w") as archivo:
        json.dump(data, archivo)
    print(f"Datos guardados exitosamente en {destino}.")
else:
    print(f"Error al descargar datos. Código de estado: {response.status_code}")




Datos guardados exitosamente en aire.
